In [1]:
# Install Gradio
!pip install gradio

In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%%capture
%cd /content/drive/MyDrive/summarization_project

In [4]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Set the project folder path
project_dir = "/content/drive/MyDrive/summarization_project"

# Define model and image paths
model_path = os.path.join(project_dir, "t5-base-finetuned")
length_image = os.path.join(project_dir, "length_comparison.png")
rouge_image = os.path.join(project_dir, "rouge_scores.png")

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [5]:
import matplotlib.pyplot as plt

# Define the summarization function
def generate_summary(article):
    if not article.strip():
        return "Error: The input text is empty. Please provide a news article."

    # Tokenize the input article
    inputs = tokenizer("summarize: " + article, return_tensors="pt", truncation=True)

    # Generate the summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Generate word count comparison bar chart
    article_length = len(article.split())
    summary_length = len(summary.split())

    plt.figure(figsize=(6, 4))
    plt.bar(["Article", "Summary"], [article_length, summary_length], color=["#a8dadc", "#457b9d"])
    plt.title("Word Count Comparison")
    plt.ylabel("Word Count")
    plt.savefig("word_count_comparison.png")
    plt.close()

    return summary, "word_count_comparison.png"

In [6]:
import gradio as gr

# Build the Gradio interface
def gradio_demo():
    with gr.Blocks() as demo:
        gr.Markdown("# ✨ T5-Base Text Summarization ✨")
        gr.Markdown(
            """
            Welcome to the T5-Base Text Summarization Demo! This tool allows you to input any news article
            and generates a concise summary using a fine-tuned T5-base model. Enjoy exploring the magic of NLP!
            """
        )

        with gr.Row():
            with gr.Column():
                article_input = gr.Textbox(
                    label="News Article",
                    placeholder="Paste your news article here...",
                    lines=10
                )
                summary_button = gr.Button("Generate Summary ✨")
                summary_output = gr.Textbox(
                    label="Generated Summary",
                    placeholder="Your summary will appear here.",
                    lines=5
                )
                word_count_plot = gr.Image(label="Word Count Comparison")

            with gr.Column():
                gr.Markdown("### Model Performance")
                gr.Markdown(
                    """
                    Below are the performance visualizations for this fine-tuned T5-base model:
                    - **Word Count Comparison** (based on a test example)
                    - **ROUGE Scores** (evaluating summary quality)
                    """
                )
                gr.Image(length_image, label="Test Example: Length Comparison")
                gr.Image(rouge_image, label="ROUGE Scores")

        summary_button.click(
            fn=generate_summary,
            inputs=article_input,
            outputs=[summary_output, word_count_plot]
        )

    return demo

In [7]:
# Launch the Gradio interface
demo = gradio_demo()
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9632e34817931fcf7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
